In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing

In [2]:
playtest5_data = pd.read_csv("2018-11-14_Playtest/anonymized_playtest5_data.csv", sep=";")
playtest6_data = pd.read_csv("2019-01-07_Playtest/anonymized_playtest6_data.csv", sep=";")
playtest7_data = pd.read_csv("2019-01-31_Playtest/anonymized_playtest7_data.csv", sep=";")

In [4]:
playtest5_data['time'] = pd.to_datetime(playtest5_data['time'])
playtest6_data['time'] = pd.to_datetime(playtest6_data['time'])
playtest7_data['time'] = pd.to_datetime(playtest7_data['time'])

In [5]:
playtest5_data = playtest5_data.sort_values('time')
playtest6_data = playtest6_data.sort_values('time')
playtest7_data = playtest7_data.sort_values('time')

In [6]:
all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])

In [7]:
all_playtest_data = all_playtest_data.sort_values('time')

In [8]:
data_by_user = all_playtest_data.groupby('session_id').agg({'id':'count',
                                             'type':'nunique'}).reset_index().rename(columns={'id':'n_events',
                                                                                              'type':'n_different_events'})

In [9]:
data_by_user['active_time'] = np.nan
data_by_user['avg_dt'] = np.nan
data_by_user['std_dt'] = np.nan

In [10]:
data_by_user.index = data_by_user['session_id'].values

In [11]:
for user in data_by_user['session_id'].unique():
    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
   
    # Computing active time
    previousEvent = None
    theresHoldActivity = 15 # np.percentile(allDifferences, 98) is 10 seconds
    activeTime = []
    
    for enum, event in user_events.iterrows():
        
        # If it is the first event
        if(previousEvent is None):
            previousEvent = event
            continue
        
        delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
        if(~(delta_seconds > theresHoldActivity)):
            activeTime.append(delta_seconds)
        
        previousEvent = event
        
    data_by_user.at[user, 'active_time'] = round(np.sum(activeTime)/60,2)
    data_by_user.at[user, 'avg_dt'] = round(np.mean(activeTime)/60,2)
    data_by_user.at[user, 'std_dt'] = round(np.std(activeTime)/60,2)